# Index
* [Import](#index)
* [Tokenization](#Tokenization)
* [CountVectorizer](#CountVectorizer)
* [PreprocessPipeline](#PreprocessPipeline)
* [OvsRest](#OvsRest)
    - [LogisticRegression](#LogisticRegression)
    - [SVC](#SVC)
* [Word2Vec](#Word2Vec)
* [BERT](#BERT)
* [USE](#USE)

In [1]:
import time
import json

# from joblib import dump, load, Memory
import joblib

# import warnings

import numpy as np
import pandas as pd
import gensim
import matplotlib.pyplot as plt

# from gensim import (
#     corpora,
# )  # https://pypi.org/project/gensim/ #https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html#sphx-glr-auto-examples-core-run-topics-and-transformations-py
from sklearn import (
    metrics,
    feature_extraction,
    linear_model,
    model_selection,
    preprocessing,
    cluster,
    decomposition,
    multiclass,
    svm,
    pipeline,
    exceptions,
    manifold,
)
from sklearn.utils._testing import ignore_warnings
from sklearn.base import BaseEstimator, TransformerMixin
from tensorflow import keras


# import nlpk

test https://scikit-learn.org/stable/modules/multiclass.html autre stategie que OneVsRest
https://keras.io/examples/nlp/text_extraction_with_bert/

In [2]:
pip install -q jupyter-black jupyter

Note: you may need to restart the kernel to use updated packages.


In [3]:
# for black formating
import jupyter_black  # pip install jupyter-black jupyter

jupyter_black.load()
# ??jupyter_black.load #show config

In [4]:
df = pd.read_csv(
    "/kaggle/input/p5-nlp/stackoverflowCleaned.csv", index_col=0, low_memory=False
)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118442 entries, 0 to 118441
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Id           118442 non-null  int64 
 1   Title        118442 non-null  object
 2   Body         118442 non-null  object
 3   0            118440 non-null  object
 4   1            118383 non-null  object
 5   2            118292 non-null  object
 6   3            118292 non-null  object
 7   4            76389 non-null   object
 8   5            4 non-null       object
 9   Title_clean  118389 non-null  object
 10  Code         88202 non-null   object
 11  Body_clean   118392 non-null  object
dtypes: int64(1), object(11)
memory usage: 11.7+ MB


In [5]:
LONG = False
description = {
    "title": "Word2Vec fix & avec pipeline custom transformer",
    "CountVectorizer": {"min_df": 0.00001, "max_df": 0.995, "max_features": None},
    "StandardScaler": {"actif": False},
    "TruncatedSVD": {"actif": False, "n_components": 1_000},
    "LogisticRegression": {"actif": False},
    "TfidfOvRSVC": {"actif": False, "max_iter": 100},
    "Word2Vec": {
        "actif": True,
        "min_count": 1,
        "size": 500,
        "window": 5,
        "epochs": 100,
        "maxlen": 30,
        "max_iter": 1_000,
    },
}

In [6]:
class colors:
    PURPLE = "\033[95m"
    BLUE = "\033[94m"
    CYAN = "\033[96m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"

# Tokenization

In [7]:
from ipywidgets import interact


@interact(i=(0, df.shape[0] - 1))
def test_tr_print(i):
    print(
        f'Title:{df["Title"][i]}\n{"*"*30}\nTitle_clean:{df["Title_clean"][i]}\n{"*"*30}\nBody:{df["Body"][i]}\n{"*"*30}\nBody_clean:{df["Body_clean"][i]}\n{"*"*30}\nCode:{df["Code"][i]}\n{"*"*30}\nTags:{df.loc[i,"0"]} {df.loc[i,"1"]} {df.loc[i,"2"]} {df.loc[i,"3"]} {df.loc[i,"4"]}'
    )

interactive(children=(IntSlider(value=59220, description='i', max=118441), Output()), _dom_classes=('widget-in…

In [8]:
sum(df["Title_clean"].isna())

53

In [9]:
df[df["Title_clean"].isna()][
    ["Title", "Title_clean", "Body", "Body_clean", "0", "1", "2", "3", "4"]
].head(5)

,Title,Title_clean,Body,Body_clean,0,1,2,3,4
302,Is < faster than <=?,NaN,<p>Is <code>if (a &lt; 901)</code> faster than...,example performance change loop code suppose g...,c++,c,performance,assembly,relational-operators
393,What is setup.py?,NaN,<p>What is <code>setup.py</code> and how can i...,configure,python,pypi,setup.py,python-packaging,NaN
1437,What is related_name used for?,NaN,<p>What is the <code>related_name</code> argum...,argument manytomanyfield foreignkey field exam...,python,django,foreign-keys,many-to-many,NaN
4062,"What is ""android:allowBackup""?",NaN,"<p>Since the <a href=""http://tools.android.com...",adt preview version version lint warning tell ...,android,adt,compiler-warnings,android-lint,android-backup-service
4870,Why is 0 < -0x80000000?,NaN,<p>I have below a simple program:</p>\n\n<pre>...,program condition if(bal int32_min work change...,c,signed,numeric-limits,numeric-conversion,NaN


In [10]:
print(sum(df["Body_clean"].isna()))
df[df["Body_clean"].isna()][
    ["Title", "Title_clean", "Body", "Body_clean", "Code"]
].head(5)

50


,Title,Title_clean,Body,Body_clean,Code
5,What and where are the stack and heap?,stack heap,<ul>\n<li>What are the stack and heap?</li>\n<...,NaN,NaN
168,What is The Rule of Three?,rule,<ul>\n<li>What does <em>copying an object</em>...,NaN,NaN
828,Why are these constructs using pre and post-in...,construct pre behavior,<pre><code>#include &lt;stdio.h&gt;\n\nint mai...,NaN,#include <stdio.h>\n\nint main(void)\n{\n in...
1207,How to use ADB Shell when Multiple Devices are...,use adb shell multiple device connect fail err...,<pre><code>$ adb --help\n</code></pre>\n<hr />...,NaN,$ adb --help\n -s SERIAL use device with give...
1640,What are the advantages of list initialization...,advantage list initialization brace,<pre><code>MyClass a1 {a}; // clearer and ...,NaN,MyClass a1 {a}; // clearer and less error-...


In [11]:
len(df[df["Body_clean"].isna() & df["Title_clean"].isna()])

0

In [12]:
sum((df["Title_clean"] + df["Body_clean"]).isna())

103

In [13]:
def tag_is_in(df: pd.DataFrame(), tag: str, nb_cols: int = 6):
    return eval("|".join(f'(df["{i}"] == "{tag}")' for i in range(0, nb_cols)))


def tags_are_in(df: pd.DataFrame(), tags: list):
    return pd.DataFrame({tag: tag_is_in(df, tag) for tag in tags})


def save_score(y_true, y_pred, target_names: list, name=None) -> pd.DataFrame():
    scores = pd.DataFrame(
        metrics.classification_report(
            y_true, y_pred, target_names=target_names, zero_division=0, output_dict=True
        )
    ).T
    if name is not None:
        scores.to_csv(f"/kaggle/working/{name}_score.csv")
    return scores


# tags_are_in(df, ["javascript","java"])
tags_are_in(
    df[10001:15050], ["javascript", "java", "c#", "python", "html", "git", "php"]
)

,javascript,java,c#,python,html,git,php
10001,True,False,False,False,True,False,False
10002,False,True,False,False,False,False,False
10003,False,False,True,False,False,False,False
10004,False,False,False,False,True,False,False
10005,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
15045,False,False,False,False,False,False,False
15046,False,False,False,False,False,False,False
15047,True,False,False,False,False,False,False
15048,False,False,False,False,False,False,False


In [14]:
tags = df[["0", "1", "2", "3", "4", "5"]].stack()
target_names = [
    tag
    for tag, freq in tags.reset_index(drop=True).value_counts(normalize=True).items()
    if freq > 0.002
]
if not LONG:
    target_names = target_names[0:20] + ["git"]
target_names

['java',
 'c#',
 'javascript',
 'python',
 'android',
 'c++',
 'ios',
 'html',
 'php',
 '.net',
 'jquery',
 'css',
 'objective-c',
 'c',
 'sql',
 'iphone',
 'asp.net',
 'mysql',
 'linux',
 'node.js',
 'git']

In [15]:
# Enregistrement des paramétres avec la liste des tags a predire
description["target_names"] = target_names
with open("/kaggle/working/description.json", "w") as outfile:
    json.dump(description, outfile)

## Separation train/test

In [16]:
df["Token"] = df["Title_clean"].fillna("") + df["Body_clean"].fillna("")
print(
    f"{len(df[[len(token.split()) > 1 for token in df['Token']]])/len(df):.5%} des questions ont plus d'un tokens"
    if len(df[df["Token"] == ""]) == 0
    else f"{len(df[df['Token'] == ''])} questions n'ont pas de token"
)
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    df["Token"], tags_are_in(df, target_names), random_state=42
)

99.99409% des questions ont plus d'un tokens


## Approche naïve pour comparer les résultat:
    - Valeur la plus fréquente (False)
    - Si le tag est présent dans la question

In [17]:
save_score(
    y_true=y_test, y_pred=y_test.replace([True], False), target_names=target_names
)

,precision,recall,f1-score,support
java,0.0,0.0,0.0,3558.0
c#,0.0,0.0,0.0,3473.0
javascript,0.0,0.0,0.0,3356.0
python,0.0,0.0,0.0,2612.0
android,0.0,0.0,0.0,2045.0
c++,0.0,0.0,0.0,1964.0
ios,0.0,0.0,0.0,1840.0
html,0.0,0.0,0.0,1755.0
php,0.0,0.0,0.0,1627.0
.net,0.0,0.0,0.0,1569.0


Avec prédire toujours vrai

In [18]:
save_score(
    y_true=y_test, y_pred=y_test.replace([False], True), target_names=target_names
)

,precision,recall,f1-score,support
java,0.120158,1.000000,0.214538,3558.0
c#,0.117287,1.000000,0.209950,3473.0
javascript,0.113336,1.000000,0.203598,3356.0
python,0.088210,1.000000,0.162120,2612.0
android,0.069062,1.000000,0.129201,2045.0
c++,0.066327,1.000000,0.124402,1964.0
ios,0.062139,1.000000,0.117007,1840.0
html,0.059269,1.000000,0.111905,1755.0
php,0.054946,1.000000,0.104168,1627.0
.net,0.052987,1.000000,0.100641,1569.0


In [19]:
test_str = X_test.str
pred_is_in = pd.DataFrame(
    {
        target_name: test_str.contains(f" {target_name} ", case=False, regex=False).values
        for target_name in target_names
    }
)
pred_is_in.T

,0,1,2,3,4,5,6,7,8,9,...,29601,29602,29603,29604,29605,29606,29607,29608,29609,29610
java,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
c#,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
javascript,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
python,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
android,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
c++,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ios,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
html,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
php,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
.net,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [20]:
score_naive_is_in = save_score(y_test, pred_is_in, target_names, "naive_is_in")
score_naive_is_in

,precision,recall,f1-score,support
java,0.766089,0.347948,0.478547,3558.0
c#,0.000000,0.000000,0.000000,3473.0
javascript,0.766812,0.261621,0.390136,3356.0
python,0.878273,0.475115,0.616646,2612.0
android,0.851163,0.447433,0.586538,2045.0
c++,0.805702,0.388493,0.524218,1964.0
ios,0.768456,0.248913,0.376026,1840.0
html,0.481022,0.375499,0.421760,1755.0
php,0.827292,0.476951,0.605068,1627.0
.net,0.536657,0.233270,0.325189,1569.0


# CountVectorizer

In [21]:
for column in y_train.columns:
    t_train = y_train[column].value_counts(normalize=True)
    t_test = y_test[column].value_counts(normalize=True)
    print(f"{colors.BOLD}{column}{colors.ENDC}")
    print(
        f"\t Sur entrainement {colors.GREEN}{t_train[True]:.2%}{colors.ENDC} positif {colors.RED}{t_train[False]:.2%}{colors.ENDC} negatif"
    )
    print(
        f"\t Sur test {colors.GREEN}{t_test[True]:.2%}{colors.ENDC} positif {colors.RED}{t_test[False]:.2%}{colors.ENDC} negatif"
    )
    print("\n")

java
	 Sur entrainement 12.05% positif 87.95% negatif
	 Sur test 12.02% positif 87.98% negatif


c#
	 Sur entrainement 11.80% positif 88.20% negatif
	 Sur test 11.73% positif 88.27% negatif


javascript
	 Sur entrainement 11.77% positif 88.23% negatif
	 Sur test 11.33% positif 88.67% negatif


python
	 Sur entrainement 8.61% positif 91.39% negatif
	 Sur test 8.82% positif 91.18% negatif


android
	 Sur entrainement 7.00% positif 93.00% negatif
	 Sur test 6.91% positif 93.09% negatif


c++
	 Sur entrainement 6.63% positif 93.37% negatif
	 Sur test 6.63% positif 93.37% negatif


ios
	 Sur entrainement 6.25% positif 93.75% negatif
	 Sur test 6.21% positif 93.79% negatif


html
	 Sur entrainement 5.85% positif 94.15% negatif
	 Sur test 5.93% positif 94.07% negatif


php
	 Sur entrainement 5.56% positif 94.44% negatif
	 Sur test 5.49% positif 94.51% negatif


.net
	 Sur entrainement 5.34% positif 94.66% negatif
	 Sur test 5.30% positif 94.70% negatif


jquery
	 Sur entrainement 5.36% positi

In [22]:
best_min_df = description["CountVectorizer"]["min_df"]
best_max_df = description["CountVectorizer"]["max_df"]
for min_df in [0.01, 0.05, 0.005, 0.0005, 0.00001] if LONG else [best_min_df]:
    for max_df in [0.95, 0.995, 0.9995] if LONG else [best_max_df]:
        print(f"\n{'*'*30}\n pour min_df:{min_df}; max_df:{max_df}")
        count_vectorizer = feature_extraction.text.CountVectorizer(
            min_df=min_df, max_df=max_df, max_features=None
        )
        #         pipe = pipeline.make_pipeline(feature_extraction.text.CountVectorizer(min_df=min_df, max_df=max_df, max_features=None), feature_extraction.text.TfidfTransformer( norm="l2", use_idf=True, smooth_idf=True, sublinear_tf=False))
        #         feat_train = pipe.transform(X_train["Token"])
        #         print(feat_train.shape)

        token_vectors = count_vectorizer.fit_transform(df["Token"])
        print(
            f"\ttokens unique:\n\t\t{token_vectors.shape[1]} dans title et body regroupé"
        )


******************************
 pour min_df:1e-05; max_df:0.995
	tokens unique:
		53999 dans title et body regroupé


avec n_components=1_000 : 10% de la variance éxpliqué

In [23]:
%%time
if LONG:
    truncSvd = decomposition.TruncatedSVD(n_components=1000).fit(
        preprocessing.StandardScaler(with_mean=False).fit_transform(token_vectors)
    )
    plt.figure(figsize=(30, 15))
    scree = truncSvd.explained_variance_ratio_ * 100
    plt.bar(np.arange(len(scree)) + 1, scree)
    plt.plot(np.arange(len(scree)) + 1, scree.cumsum(), c="red", marker="o")
    plt.xlabel("rang de l'axe d'inertie")
    plt.ylabel("pourcentage variance expliquée")
    plt.title("Eboulis des valeurs propres")
    plt.show()

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs


# PreprocessPipeline

In [24]:
pipeline_tfidf = pipeline.Pipeline(
    [
        (
            "count",
            feature_extraction.text.CountVectorizer(
                min_df=best_min_df, max_df=best_max_df, max_features=None
            ),
        ),
        (
            "tfidf",
            feature_extraction.text.TfidfTransformer(
                norm="l2", use_idf=True, smooth_idf=True, sublinear_tf=False
            ),
        ),
        (
            "scale",
            preprocessing.StandardScaler(with_mean=False)
            if description["StandardScaler"]["actif"]
            else "passthrough",
        ),
        (
            "dimReduction",
            decomposition.TruncatedSVD(
                n_components=description["TruncatedSVD"]["n_components"]
            )
            if description["TruncatedSVD"]["actif"]
            else "passthrough",
        ),
    ]
)
## @TODO cache
# memory = joblib.Memory(location="/kaggle/working/cache_dir", verbose=0)
# pipeline_tfidf = memory.cache(pipeline_tfidf)

# OvsRest

## LogisticRegression

In [25]:
%%time
if description["LogisticRegression"]["actif"]:
    pipeline_logisticReg = pipeline.Pipeline(
        [
            ("tfidf", pipeline_tfidf),
            (
                "OvRLogisticReg",
                multiclass.OneVsRestClassifier(
                    linear_model.LogisticRegression(random_state=42)
                ),
            ),
        ]
    )
    with ignore_warnings(category=exceptions.ConvergenceWarning):
        pipeline_logisticReg.fit(X_train, y_train)

    # prediction sur jeu de test
    pred_lr = pipeline_logisticReg.predict(X_test)
    scores_lr = save_score(y_test, pred_lr, target_names, "LogisticRegression")

    # enregistrement model et score
    joblib.dump(pipeline_logisticReg, "LogisticRegression_model.joblib")
    scores_lr.sort_values(by="support", ascending=False)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.78 µs


In [26]:
if description["LogisticRegression"]["actif"]:
    print(scores_lr.sort_values(by="precision", ascending=False))

In [27]:
if description["LogisticRegression"]["actif"]:
    print(scores_lr.sort_values(by="recall", ascending=False))

## SVC

In [28]:
pipeline_svc = pipeline.Pipeline(
    [
        ("tfidf", pipeline_tfidf),
        (
            "TfidfOvRSVC",
            multiclass.OneVsRestClassifier(
                svm.SVC(
                    cache_size=1080,
                    max_iter=description["TfidfOvRSVC"]["max_iter"],
                    random_state=42,
                )
            ),
        ),
    ]
)

In [29]:
%%time
if description["TfidfOvRSVC"]["actif"]:
    with ignore_warnings(category=exceptions.ConvergenceWarning):
        pipeline_svc.fit(
            X_train,
            y_train,
        )

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.06 µs


In [30]:
%%time
if description["TfidfOvRSVC"]["actif"]:
    pred_svc = pipeline_svc.predict(X_test)
    print(pred_svc)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.54 µs


### Score et enregistrement

In [31]:
if description["TfidfOvRSVC"]["actif"]:
    scores_svc = save_score(y_test, pred_svc, target_names, "TfidfOvRSVC")
    joblib.dump(pipeline_svc, "TfidfOvRestSvc_model.joblib")
    print(scores_svc.sort_values(by="precision", ascending=False))

# Word2Vec

In [32]:
def create_embeding(word_index, model_vectors):
    vocab_size = len(word_index) + 1
    print(f"Number of unique words: {vocab_size}")
    print("Create Embedding matrix ...")
    embedding_matrix = np.zeros((vocab_size, description["Word2Vec"]["size"]))
    i = 0
    j = 0

    for word, idx in word_index.items():
        i += 1
        if word in model_vectors.index_to_key:
            j += 1
            embedding_vector = model_vectors[word]
            if embedding_vector is not None:
                embedding_matrix[idx] = model_vectors[word]

    word_rate = np.round(j / i, 4)
    print("Word embedding rate : ", word_rate)
    print("Embedding matrix: %s" % str(embedding_matrix.shape))
    return (embedding_matrix, vocab_size)


def create_keras_model(X_train, description=description):
    print("Build & train Word2Vec model ...")
    X_train_token = X_train.str.split()
    w2v_model = gensim.models.Word2Vec(
        min_count=description["Word2Vec"]["min_count"],
        window=description["Word2Vec"]["window"],
        vector_size=description["Word2Vec"]["size"],
        seed=42,
        workers=1,
    )
    w2v_model.build_vocab(X_train_token)
    w2v_model.train(
        X_train_token,
        total_examples=w2v_model.corpus_count,
        epochs=description["Word2Vec"]["epochs"],
    )
    model_vectors = w2v_model.wv
    print("Vocabulary size: %i" % len(model_vectors.index_to_key))
    print("Word2Vec trained")

    tokenizer = keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(X_train_token)
    embedding_matrix, vocab_size = create_embeding(tokenizer.word_index, model_vectors)

    word_input = keras.layers.Input(
        shape=(description["Word2Vec"]["maxlen"],), dtype="float64"
    )
    word_embedding = keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=description["Word2Vec"]["size"],
        weights=[embedding_matrix],
        input_length=description["Word2Vec"]["maxlen"],
    )(word_input)
    word_vec = keras.layers.GlobalAveragePooling1D()(word_embedding)
    embed_model = keras.models.Model([word_input], word_vec)
    print(embed_model.summary())

    return (embed_model, tokenizer)


class KerasEmbedTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.embed_model, self.tokenizer = create_keras_model(X)
        return self

    def transform(self, X, y=None):
        x_sentences = keras.preprocessing.sequence.pad_sequences(
            self.tokenizer.texts_to_sequences(X),
            maxlen=description["Word2Vec"]["maxlen"],
            padding="post",
        )
        embeddings = self.embed_model.predict(x_sentences)
        print("embedings shape ", embeddings.shape)
        return embeddings

In [33]:
%%time
pipeline_w2v = pipeline.Pipeline(
    [
        ("keras_embed_transformer", KerasEmbedTransformer()),
        (
            "Word2Vec",
            multiclass.OneVsRestClassifier(
                svm.SVC(
                    cache_size=1080,
                    max_iter=description["Word2Vec"]["max_iter"],
                    random_state=42,
                )
            ),
        ),
    ]
)
# pipeline_w2v.fit_transform(X_train[0:10])
if description["Word2Vec"]["actif"]:
    with ignore_warnings(category=exceptions.ConvergenceWarning):
        pipeline_w2v.fit(X_train, y_train)

Build & train Word2Vec model ...
Vocabulary size: 210897
Word2Vec trained
Number of unique words: 210898
Create Embedding matrix ...
Word embedding rate :  1.0
Embedding matrix: (210898, 500)


2022-12-25 13:57:33.302197: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 30, 500)           105449000 
_________________________________________________________________
global_average_pooling1d (Gl (None, 500)               0         
Total params: 105,449,000
Trainable params: 105,449,000
Non-trainable params: 0
_________________________________________________________________
None


2022-12-25 13:57:39.026058: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


embedings shape  (88831, 500)
CPU times: user 1h 19min 4s, sys: 28.7 s, total: 1h 19min 33s
Wall time: 1h 19min 25s


In [34]:
if description["Word2Vec"]["actif"]:
    pred_word2vec = pipeline_w2v.predict(X_test)
    print(pred_word2vec)

embedings shape  (29611, 500)
[[1 0 1 ... 0 1 0]
 [1 0 1 ... 0 1 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [1 1 1 ... 0 1 0]]


In [35]:
if description["Word2Vec"]["actif"]:
    scores_word2vec = save_score(y_test,pred_word2vec,target_names, "Word2Vec")
#     joblib.dump(pipeline_w2v, "Word2Vec_model.joblib")
    print(scores_word2vec.sort_values(by="precision", ascending=False))

              precision    recall  f1-score  support
git            0.888889  0.642336  0.745763    274.0
c#             0.170928  0.781457  0.280502   3473.0
java           0.137253  0.926363  0.239083   3558.0
c++            0.125593  0.889511  0.220108   1964.0
macro avg      0.120256  0.860981  0.176163  33044.0
javascript     0.118817  0.991359  0.212201   3356.0
python         0.112835  0.933002  0.201322   2612.0
weighted avg   0.105492  0.893142  0.180346  33044.0
samples avg    0.102949  0.711964  0.167405  33044.0
micro avg      0.085905  0.893142  0.156735  33044.0
android        0.084758  0.989242  0.156138   2045.0
ios            0.083770  0.886957  0.153081   1840.0
css            0.079436  0.956676  0.146692   1131.0
jquery         0.076457  0.960338  0.141638   1538.0
html           0.072999  0.874074  0.134745   1755.0
objective-c    0.071724  0.799550  0.131640    888.0
c              0.068146  0.780791  0.125351    885.0
php            0.068139  0.955747  0.127209   

# BERT

# LDA 

# USE